# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,petropavlovsk-kamchatsky,53.0452,158.6483,275.48,44,0,1.00,RU,1712098746
1,1,port-aux-francais,-49.3500,70.2167,280.07,79,89,13.05,TF,1712098746
2,2,nepalgunj,28.0500,81.6167,293.46,22,56,1.69,NP,1712098746
3,3,hawaiian paradise park,19.5933,-154.9731,298.88,78,75,6.17,US,1712098747
4,4,la passe,45.5549,-0.8967,284.28,86,100,8.35,FR,1712098747


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
total_map_plot = city_data_df.hvplot.points('Lng',
                                            'Lat', 
                                            geo = True, 
                                            tiles = 'OSM',
                                            frame_width = 700,
                                            frame_height = 500,
                                            size = 'Humidity',
                                            color = 'City')

# Display the map
total_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp'] < 300.15) & (city_data_df['Max Temp'] > 294.15) & 
                            (city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 4.5),:]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,76,newman,37.3138,-121.0208,298.29,40,0,1.34,US,1712098715
83,83,gebeit,21.0667,36.3167,294.97,59,0,2.49,SD,1712098753
88,88,emerald,-23.5333,148.1667,298.34,74,0,3.86,AU,1712098731
266,266,caldwell,43.6629,-116.6874,295.92,34,0,2.06,US,1712098578
349,349,puerto penasco,31.3167,-113.5333,294.21,62,0,4.47,MX,1712098776
367,367,benghazi,32.1167,20.0667,294.29,46,0,4.37,LY,1712098778
409,409,arraial do cabo,-22.9661,-42.0278,297.13,88,0,1.54,BR,1712098548
434,434,cao bang,22.6667,106.2500,294.78,96,0,0.60,VN,1712098784
505,505,hajin,34.6927,40.8270,295.44,35,0,1.45,SY,1712098792
507,507,al burayqah,30.4062,19.5739,294.65,53,0,3.21,LY,1712098792


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.loc[:,["City","Country","Lat","Lng","Humidity"]]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
76,newman,US,37.3138,-121.0208,40,
83,gebeit,SD,21.0667,36.3167,59,
88,emerald,AU,-23.5333,148.1667,74,
266,caldwell,US,43.6629,-116.6874,34,
349,puerto penasco,MX,31.3167,-113.5333,62,
367,benghazi,LY,32.1167,20.0667,46,
409,arraial do cabo,BR,-22.9661,-42.0278,88,
434,cao bang,VN,22.6667,106.2500,96,
505,hajin,SY,34.6927,40.8270,35,
507,al burayqah,LY,30.4062,19.5739,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey":geoapify_key 
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index,'Lat']
    lon = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
caldwell - nearest hotel: Best Western Plus Caldwell Inn & Suites
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
benghazi - nearest hotel: ترجاب
arraial do cabo - nearest hotel: No hotel found
cao bang - nearest hotel: SONG HIEN HOTEL
hajin - nearest hotel: No hotel found
al burayqah - nearest hotel: فندق ماس ليبيا
arafat - nearest hotel: ابراج فجر الايمان


,City,Country,Lat,Lng,Humidity,Hotel Name
76,newman,US,37.3138,-121.0208,40,No hotel found
83,gebeit,SD,21.0667,36.3167,59,No hotel found
88,emerald,AU,-23.5333,148.1667,74,Emerald Central Hotel
266,caldwell,US,43.6629,-116.6874,34,Best Western Plus Caldwell Inn & Suites
349,puerto penasco,MX,31.3167,-113.5333,62,Hotel Paraíso Del Desierto
367,benghazi,LY,32.1167,20.0667,46,ترجاب
409,arraial do cabo,BR,-22.9661,-42.0278,88,No hotel found
434,cao bang,VN,22.6667,106.2500,96,SONG HIEN HOTEL
505,hajin,SY,34.6927,40.8270,35,No hotel found
507,al burayqah,LY,30.4062,19.5739,53,فندق ماس ليبيا


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points('Lng',
                                        'Lat', 
                                        geo = True, 
                                        tiles = 'OSM',
                                        frame_width = 700,
                                        frame_height = 500,
                                        size = 'Humidity',
                                        color = 'City',
                                        hover_cols = ['Hotel Name','Country'])

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)